<a href="https://colab.research.google.com/github/anindabitm/Malawi_flood_prediction/blob/master/Flood_prediction_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/anindabitm/Malawi_flood_prediction/master/Train_1.csv'
data=pd.read_csv(url,index_col='Square_ID')

In [0]:
print(data.shape)
data.head(5)

(16466, 39)


,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,precip 2014-12-28 - 2015-01-04,precip 2015-01-04 - 2015-01-11,precip 2015-01-11 - 2015-01-18,precip 2015-01-18 - 2015-01-25,precip 2015-01-25 - 2015-02-01,precip 2015-02-01 - 2015-02-08,precip 2015-02-08 - 2015-02-15,precip 2015-02-15 - 2015-02-22,precip 2015-02-22 - 2015-03-01,precip 2015-03-01 - 2015-03-08,precip 2015-03-08 - 2015-03-15,precip 2019-01-20 - 2019-01-27,precip 2019-01-27 - 2019-02-03,precip 2019-02-03 - 2019-02-10,precip 2019-02-10 - 2019-02-17,precip 2019-02-17 - 2019-02-24,precip 2019-02-24 - 2019-03-03,precip 2019-03-03 - 2019-03-10,precip 2019-03-10 - 2019-03-17,precip 2019-03-17 - 2019-03-24,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode
Square_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4e3c3896-14ce-11ea-bce5-f49634744a41,34.26,-15.91,0.0,887.764222,0.0,0.0,0.0,14.844025,14.552823,12.237766,57.451361,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,12.99262,4.582856,35.037532,4.796012,28.083314,0.0,58.362456,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9
4e3c3897-14ce-11ea-bce5-f49634744a41,34.26,-15.90,0.0,743.403912,0.0,0.0,0.0,14.844025,14.552823,12.237766,57.451361,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,12.99262,4.582856,35.037532,4.796012,28.083314,0.0,58.362456,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9
4e3c3898-14ce-11ea-bce5-f49634744a41,34.26,-15.89,0.0,565.728343,0.0,0.0,0.0,14.844025,14.552823,12.237766,57.451361,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,12.99262,4.582856,35.037532,4.796012,28.083314,0.0,58.362456,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9
4e3c3899-14ce-11ea-bce5-f49634744a41,34.26,-15.88,0.0,443.392774,0.0,0.0,0.0,14.844025,14.552823,12.237766,57.451361,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,12.99262,4.582856,35.037532,4.796012,28.083314,0.0,58.362456,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10
4e3c389a-14ce-11ea-bce5-f49634744a41,34.26,-15.87,0.0,437.443428,0.0,0.0,0.0,14.844025,14.552823,12.237766,57.451361,30.127047,30.449468,1.521829,29.389995,32.878318,8.179804,0.963981,16.659097,3.304466,0.0,12.99262,4.582856,35.037532,4.796012,28.083314,0.0,58.362456,18.264692,17.537486,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10


In [0]:
cols=data.columns
print("The columns in the data are:\n")
print(cols)

The columns in the data are:

Index(['X', 'Y', 'target_2015', 'elevation', 'precip 2014-11-16 - 2014-11-23',
       'precip 2014-11-23 - 2014-11-30', 'precip 2014-11-30 - 2014-12-07',
       'precip 2014-12-07 - 2014-12-14', 'precip 2014-12-14 - 2014-12-21',
       'precip 2014-12-21 - 2014-12-28', 'precip 2014-12-28 - 2015-01-04',
       'precip 2015-01-04 - 2015-01-11', 'precip 2015-01-11 - 2015-01-18',
       'precip 2015-01-18 - 2015-01-25', 'precip 2015-01-25 - 2015-02-01',
       'precip 2015-02-01 - 2015-02-08', 'precip 2015-02-08 - 2015-02-15',
       'precip 2015-02-15 - 2015-02-22', 'precip 2015-02-22 - 2015-03-01',
       'precip 2015-03-01 - 2015-03-08', 'precip 2015-03-08 - 2015-03-15',
       'precip 2019-01-20 - 2019-01-27', 'precip 2019-01-27 - 2019-02-03',
       'precip 2019-02-03 - 2019-02-10', 'precip 2019-02-10 - 2019-02-17',
       'precip 2019-02-17 - 2019-02-24', 'precip 2019-02-24 - 2019-03-03',
       'precip 2019-03-03 - 2019-03-10', 'precip 2019-03-10 - 2019

In [0]:
data.describe()

,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,precip 2014-12-28 - 2015-01-04,precip 2015-01-04 - 2015-01-11,precip 2015-01-11 - 2015-01-18,precip 2015-01-18 - 2015-01-25,precip 2015-01-25 - 2015-02-01,precip 2015-02-01 - 2015-02-08,precip 2015-02-08 - 2015-02-15,precip 2015-02-15 - 2015-02-22,precip 2015-02-22 - 2015-03-01,precip 2015-03-01 - 2015-03-08,precip 2015-03-08 - 2015-03-15,precip 2019-01-20 - 2019-01-27,precip 2019-01-27 - 2019-02-03,precip 2019-02-03 - 2019-02-10,precip 2019-02-10 - 2019-02-17,precip 2019-02-17 - 2019-02-24,precip 2019-02-24 - 2019-03-03,precip 2019-03-03 - 2019-03-10,precip 2019-03-10 - 2019-03-17,precip 2019-03-17 - 2019-03-24,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode
count,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.000000,16466.00000
mean,35.077656,-15.813802,0.076609,592.848206,1.610760,2.502058,1.162076,8.270610,8.892459,9.572821,22.925036,28.113210,58.859208,1.251173,34.653177,28.314888,12.487909,3.802584,17.072285,9.110949,0.330641,13.329023,4.437490,23.149500,9.749785,29.575991,1.864399,60.424964,12.321620,35.637354,2.126234,3.453395,3.559366,9.127677,1.660709,0.526144,0.968101,1.585743,10.73175
std,0.392395,0.359789,0.228734,354.790357,4.225461,8.631846,4.396676,4.263375,3.760052,4.523767,13.690451,7.794291,16.807838,1.969923,7.456422,8.047223,7.064435,2.674434,6.074926,4.572201,1.008490,5.552818,5.163184,6.148509,4.558172,8.508608,4.158313,8.313199,9.900994,14.519169,3.734829,8.007248,3.820294,6.868937,4.418032,1.494935,3.690698,4.651863,2.02610
min,34.260000,-16.640000,0.000000,45.541444,0.000000,0.000000,0.000000,1.411452,3.580342,1.254098,7.462999,15.648154,30.449468,0.000000,14.964383,13.261280,0.459067,0.279002,6.728685,3.283425,0.000000,3.813864,0.000000,12.450319,2.801546,12.780855,0.000000,32.108226,0.000000,15.803429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.00000
25%,34.760000,-16.070000,0.000000,329.063852,0.000000,0.000000,0.000000,5.548440,5.905440,6.179885,11.617057,23.483879,45.972601,0.000000,30.037450,22.262417,5.090802,1.654155,13.769957,5.538848,0.000000,8.946479,0.888216,19.590606,6.761704,25.360910,0.000000,56.380490,3.120173,22.021763,0.000000,0.000000,0.000000,4.352528,0.000000,0.000000,0.000000,0.000000,9.00000
50%,35.050000,-15.800000,0.000000,623.000000,0.000000,0.000000,0.000000,7.941822,8.618390,8.786780,18.381539,26.085586,55.501115,0.502164,34.363729,26.512675,14.092012,3.301029,15.549508,8.235819,0.000000,12.913147,2.249833,21.661698,9.076457,30.593415,0.690000,60.581696,12.508606,34.275716,0.896323,0.000000,2.607053,7.862453,0.000000,0.000000,0.000000,0.000000,10.00000
75%,35.390000,-15.520000,0.000000,751.434813,1.261848,0.000000,0.000000,10.887235,10.960668,12.670775,31.304699,33.587434,69.311540,1.195866,36.715386,34.880240,18.681926,5.120276,19.836449,11.308650,0.000000,17.123831,6.832768,24.213460,10.830000,33.300457,1.429370,65.721446,20.004375,44.253897,2.076590,2.914996,6.390000,13.459070,0.000000,0.000000,0.000000,0.000000,12.00000
max,35.860000,-15.210000,1.000000,2803.303645,19.354969,41.023858,22.020803,18.870675,23.044340,21.757828,62.433432,51.197420,105.275891,11.103658,53.014243,44.341312,28.559

In [0]:
data.isnull().sum()

X                                 0
Y                                 0
target_2015                       0
elevation                         0
precip 2014-11-16 - 2014-11-23    0
precip 2014-11-23 - 2014-11-30    0
precip 2014-11-30 - 2014-12-07    0
precip 2014-12-07 - 2014-12-14    0
precip 2014-12-14 - 2014-12-21    0
precip 2014-12-21 - 2014-12-28    0
precip 2014-12-28 - 2015-01-04    0
precip 2015-01-04 - 2015-01-11    0
precip 2015-01-11 - 2015-01-18    0
precip 2015-01-18 - 2015-01-25    0
precip 2015-01-25 - 2015-02-01    0
precip 2015-02-01 - 2015-02-08    0
precip 2015-02-08 - 2015-02-15    0
precip 2015-02-15 - 2015-02-22    0
precip 2015-02-22 - 2015-03-01    0
precip 2015-03-01 - 2015-03-08    0
precip 2015-03-08 - 2015-03-15    0
precip 2019-01-20 - 2019-01-27    0
precip 2019-01-27 - 2019-02-03    0
precip 2019-02-03 - 2019-02-10    0
precip 2019-02-10 - 2019-02-17    0
precip 2019-02-17 - 2019-02-24    0
precip 2019-02-24 - 2019-03-03    0
precip 2019-03-03 - 2019-03-

In [0]:
predict_cols=['X','Y','elevation','precip 2014-11-16 - 2014-11-23',
       'precip 2014-11-23 - 2014-11-30', 'precip 2014-11-30 - 2014-12-07',
       'precip 2014-12-07 - 2014-12-14', 'precip 2014-12-14 - 2014-12-21',
       'precip 2014-12-21 - 2014-12-28', 'precip 2014-12-28 - 2015-01-04',
       'precip 2015-01-04 - 2015-01-11', 'precip 2015-01-11 - 2015-01-18',
       'precip 2015-01-18 - 2015-01-25', 'precip 2015-01-25 - 2015-02-01',
       'precip 2015-02-01 - 2015-02-08', 'precip 2015-02-08 - 2015-02-15',
       'precip 2015-02-15 - 2015-02-22', 'precip 2015-02-22 - 2015-03-01',
       'precip 2015-03-01 - 2015-03-08', 'precip 2015-03-08 - 2015-03-15','LC_Type1_mode']
label_cols='target_2015'
df=data.loc[:,predict_cols + [label_cols]]
df.columns=['X','Y','elevation','1W','2W','3W','4W','5W','6W','7W','8W','9W','10W','11W','12W','13W','14W','15W','16W','17W','LC_Type1_mode','target']
predict_cols=['X','Y','elevation','1W','2W','3W','4W','5W','6W','7W','8W','9W','10W','11W','12W','13W','14W','15W','16W','17W','LC_Type1_mode']
label_cols='target'
print("Shape of df is:",df.shape)
print(df.columns)
X=df.loc[:,predict_cols]
y=df.loc[:,label_cols]

Shape of df is: (16466, 22)
Index(['X', 'Y', 'elevation', '1W', '2W', '3W', '4W', '5W', '6W', '7W', '8W',
       '9W', '10W', '11W', '12W', '13W', '14W', '15W', '16W', '17W',
       'LC_Type1_mode', 'target'],
      dtype='object')


In [0]:
predict_test_cols=['X','Y','elevation','precip 2019-01-20 - 2019-01-27', 'precip 2019-01-27 - 2019-02-03',
       'precip 2019-02-03 - 2019-02-10', 'precip 2019-02-10 - 2019-02-17',
       'precip 2019-02-17 - 2019-02-24', 'precip 2019-02-24 - 2019-03-03',
       'precip 2019-03-03 - 2019-03-10', 'precip 2019-03-10 - 2019-03-17',
       'precip 2019-03-17 - 2019-03-24', 'precip 2019-03-24 - 2019-03-31',
       'precip 2019-03-31 - 2019-04-07', 'precip 2019-04-07 - 2019-04-14',
       'precip 2019-04-14 - 2019-04-21', 'precip 2019-04-21 - 2019-04-28',
       'precip 2019-04-28 - 2019-05-05', 'precip 2019-05-05 - 2019-05-12',
       'precip 2019-05-12 - 2019-05-19','LC_Type1_mode']
df_test=data.loc[:,predict_test_cols]
df_test.columns=['X','Y','elevation','1W','2W','3W','4W','5W','6W','7W','8W','9W','10W','11W','12W','13W','14W','15W','16W','17W','LC_Type1_mode']
print("Shape of df_test is:",df_test.shape)

Shape of df_test is: (16466, 21)


In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
rf=RandomForestRegressor(n_estimators=10)
ada=AdaBoostRegressor()
gb=GradientBoostingRegressor()
dtr=DecisionTreeRegressor()
#cross_val_scores = cross_val_score(rf,X,y,cv=10,scoring='neg_mean_squared_error')
#print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.50,random_state=123)
rf.fit(X_train,y_train)
rf_predict=rf.predict(X_valid)
ada.fit(X_train,y_train)
ada_predict=ada.predict(X_valid)
gb.fit(X_train,y_train)
gb_predict=gb.predict(X_valid)
dtr.fit(X_train,y_train)
dtr_predict=dtr.predict(X_valid)
print('RMSE for validation set through RF is:',np.mean(np.sqrt(mean_squared_error(y_valid,rf_predict))))
X_valid.loc[:,'rf_preds']=rf_predict.tolist()
print('RMSE for validation set through Ada boost is:',np.mean(np.sqrt(mean_squared_error(y_valid,ada_predict))))
X_valid.loc[:,'ada_preds']=ada_predict.tolist()
print('RMSE for validation set through Gradient boost is:',np.mean(np.sqrt(mean_squared_error(y_valid,gb_predict))))
X_valid.loc[:,'gb_preds']=gb_predict.tolist()
print('RMSE for validation set through Decision Tree Regressor is:',np.mean(np.sqrt(mean_squared_error(y_valid,dtr_predict))))
X_valid.loc[:,'dtr_preds']=dtr_predict.tolist()
print(X_valid.shape)

RMSE for validation set through RF is: 0.10251543004696663
RMSE for validation set through Ada boost is: 0.15690794452020232
RMSE for validation set through Gradient boost is: 0.11817061169707974
RMSE for validation set through Decision Tree Regressor is: 0.1313440028399106
(8233, 25)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [0]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
gbm_param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
        }

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(n_estimators=100,
                    silent=True, nthread=1)

# Perform grid search: grid_mse
rnd_mse = RandomizedSearchCV(estimator=gbm,param_distributions=gbm_param_grid,scoring='neg_mean_squared_error',cv=4,verbose=0)


# Fit grid_mse to the data
rnd_mse.fit(X_valid,y_valid)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", rnd_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(rnd_mse.best_score_)))


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/x

Best parameters found:  {'subsample': 0.6, 'min_child_weight': 10, 'max_depth': 3, 'learning_rate': 0.3, 'gamma': 0.5, 'colsample_bytree': 0.8}
Lowest RMSE found:  0.09983631700350998


In [0]:
rf_test=rf.predict(df_test)
ada_test=ada.predict(df_test)
gb_test=gb.predict(df_test)
dtr_test=dtr.predict(df_test)
df_test.loc[:,'rf_preds']=rf_test.tolist()
df_test.loc[:,'ada_preds']=ada_test.tolist()
df_test.loc[:,'gb_preds']=gb_test.tolist()
df_test.loc[:,'dtr_preds']=dtr_test.tolist()

In [0]:
predictions=rnd_mse.predict(df_test)

In [0]:
url_sub='https://raw.githubusercontent.com/anindabitm/Malawi_flood_prediction/master/SampleSubmission_1.csv'
submission=pd.read_csv(url_sub)
print(submission.head())

                              Square_ID  target_2019
0  4e3c3896-14ce-11ea-bce5-f49634744a41        0.109
1  4e3c3897-14ce-11ea-bce5-f49634744a41        0.504
2  4e3c3898-14ce-11ea-bce5-f49634744a41        0.021
3  4e3c3899-14ce-11ea-bce5-f49634744a41        0.827
4  4e3c389a-14ce-11ea-bce5-f49634744a41        0.786


In [0]:
submission.loc[:,'target_2019']=predictions
print(submission.shape)
submission.to_csv('results.csv',index=False)

(16466, 2)


In [0]:
from google.colab import files
files.download('results.csv') 